In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 데이터 저장할 폴더 경로
data_folder = "./data"
os.makedirs(data_folder, exist_ok=True)  # 폴더가 없으면 생성

# 1부터 10 페이지까지 반복
for page in range(20, 21):
    url = f"https://www.koreapas.com/m/sofo_result.php?salt=1&place=&continue=1&page={page}"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    # 페이지별 파일 경로 설정
    page_file_path = os.path.join(data_folder, f'page{page}.txt')
    
    # 페이지별 파일 생성
    with open(page_file_path, 'w', encoding='utf-8') as file:
        # 'ani' 클래스를 가진 모든 span 태그 찾기
        elements = driver.find_elements(By.CSS_SELECTOR, 'span.ani[style="font-weight:700;letter-spacing:-1px;font-size:16px;"]')
        
        for element in elements[-15:]: 
            # 클릭하기 전에 스크롤을 해당 요소 위치까지 이동
            driver.execute_script("arguments[0].click();", element)
            time.sleep(1)  # 클릭 후 상세 페이지 로딩 대기
            
            # 상세 페이지에서 데이터 추출
            try:
                # 첫 번째 span 태그 데이터 추출
                first_span_text = driver.find_element(By.CSS_SELECTOR, 'span[style="font-size:7.5vw;font-weight:bold;color:#fff;text-shadow: 1px 1px 1px black;"]').text
                file.write(f"식당이름:<{first_span_text}>\n")
                
                # 두 번째 span에서 태그 데이터 추출
                second_span_text = driver.find_elements(By.CSS_SELECTOR, 'span[style="color:#000;font-size:14px;"]')[0].text
                file.write(f"[위치]:{second_span_text}\n")
                # 세 번째 span 태그 데이터 추출
                third_span_text = driver.find_elements(By.CSS_SELECTOR, 'span[style="color:#000;font-size:14px;"]')[1].text
                file.write(f"[운영시간]:{third_span_text}\n")
                
                # 네 번째 span 태그 데이터 추출 (최대 10개)
                fourth_span_elements = driver.find_elements(By.CSS_SELECTOR, 'span[style="letter-spacing:-0.5px;font-size:16px;color:#000;word-break:keep-all;"]')
                for fourth_span in fourth_span_elements[:10]:
                    file.write(f"식당이름:<{first_span_text}>, 리뷰:{fourth_span.text}\n")
                file.write("\n")  # 각 섹션 후에 공백 줄 추가
            except Exception as e:
                print(f"에러 발생: {e}")
            
            # 이전 페이지로 돌아가기
            driver.back()
            time.sleep(2)  # 이전 페이지 로딩 대기

# 드라이버 종료
driver.quit()
